# Data Scraping

## Scraping Twitter data

Here I scrape tweets from @nytimes made between 2022-01-01 and mid-April 2022 using the Twitter API, twint, and snscrape.

In [ ]:
''' Getting tweets from @nytimes using the Twitter API v2.
    Unfortunately, I was only able to get 30 days of tweets using this method.
'''

import requests
import os
import time
import json

twitterAPI = 'https://api.twitter.com/2/users/'
nytUserID = '807095' # Twitter ID for the New York Times
startTime = '2022-01-01T00:00:00.000Z' # Beginning of the year
endTime = '2022-04-30T00:00:00.000Z' # End of April

resultingJSON = {'meta': {'next_token': ''}}
while True:
    try:
        pagination_token = resultingJSON['meta']['next_token']
        if pagination_token != '':
            pagination_token = 'pagination_token=' + pagination_token
    except KeyError:
        break
    fields = ('/tweets?max_results=100&' +
                pagination_token +
                '&start_time=' + startTime +
                '&end_time=' + endTime +
                '&tweet.fields=id,created_at,text,author_id,in_reply_to_user_id,referenced_tweets,attachments,geo,entities,public_metrics,source,context_annotations,conversation_id&media.fields=media_key,duration_ms,height,preview_image_url,type,url,width,public_metrics,alt_text'
    )
    URL = twitterAPI + nytUserID + fields

    req = requests.get(URL, headers = {'Authorization': f'Bearer {os.environ["BEARER_TOKEN"]}'})
    resultingJSON = req.json()
    
    with open('nyt_' + pagination_token + '.json', 'w') as file:
        json.dump(resultingJSON, file)

    time.sleep(10) # To keep the number of requests well below the Twitter rate limit

In [ ]:
''' Getting tweets from @nytimes using twint.
    Unfortunately, I was only able to get 10 days of tweets using this method.  Moreover, retweets are missing.
'''

import twint
config = twint.Config()
config.Username = 'nytimes'
config.Since = '2022-01-01'
config.Store_json = True
config.Output = 'nyt_data_twint.json'
config.Retweets = True

twint.run.Search(config)

In [ ]:
''' Getting tweets from @nytimes using snscrape, on command line.
    This method seems to be able to get tweets from Jan 2022, unlike the Twitter API and twint.  However, retweets are missing.
'''

# snscrape --jsonl --progress --since 2022-01-01 twitter-search "from:nytimes until:2022-04-30" >> nyt_twitter_data.json

## Loading Twitter data into a postgreSQL database

In [ ]:
import psycopg2
from psycopg2 import sql
from collections import OrderedDict
import json

conn = psycopg2.connect(host = 'localhost')
conn.autocommit = True
cursor = conn.cursor()

# Create database
cursor.execute('CREATE DATABASE nytpopular;')

conn.close()
cursor.close()

conn = psycopg2.connect(host = 'localhost', database = 'nytpopular')
conn.autocommit = True
cursor = conn.cursor()

# Create tweets table
cursor.execute('''CREATE TABLE tweets (
	id BIGINT PRIMARY KEY,
    retweet_count INT,
    reply_count INT,
    like_count INT,
    quote_count INT,
    url VARCHAR(255),
    text TEXT,
    date TIMESTAMPTZ,
    video BOOLEAN,
    photo BOOLEAN
);''')

def insertTweetintoDB(jsonObject, cursor):
    fields = OrderedDict()
    fields['id'] = jsonObject['id']
    fields['retweet_count'] = jsonObject['retweetCount']
    fields['reply_count'] = jsonObject['replyCount']
    fields['like_count'] = jsonObject['likeCount']
    fields['quote_count'] = jsonObject['quoteCount']
    if jsonObject['outlinks'] is None:
        return
    fields['url'] = ' '.join(jsonObject['outlinks'])
    fields['text'] = jsonObject['content']
    fields['date'] = jsonObject['date'].replace('T',' ')
    if ('media' in jsonObject) and (jsonObject['media'] is not None):
        containsPhoto = any(['Photo' in elem['_type'] for elem in jsonObject['media']])
        containsVideo = any(['Video' in elem['_type'] for elem in jsonObject['media']])
    else:
        containsPhoto = False
        containsVideo = False
    fields['video'] = 'TRUE' if containsVideo else 'FALSE'
    fields['photo'] = 'TRUE' if containsPhoto else 'FALSE'
    keys, items = zip(*fields.items())
    keys = sql.SQL(',').join(map(sql.Identifier, keys))
    items = sql.Literal(items)
    command = sql.SQL('INSERT INTO tweets ({}) VALUES {} ON CONFLICT (id) DO NOTHING;').format(keys, items)
    cursor.execute(command)

with open('ScrappedData/snscrape/nyt_twitter_data.json','r') as f:
    for line in f:
        insertTweetintoDB(json.loads(line), cursor)

cursor.execute("DELETE FROM tweets WHERE text ~ '^[^a-zA-z0-9\u201c\u201d\"@#]'") # Drop some foreign language tweets from the table.
cursor.close()
conn.close()

## Scraping New York Times articles using Selenium

In [ ]:
# Define a context manager for graceful stopping a program when hitting CTRL + C.
# I am defining this so that I can pause scraping whenever I want.

from contextlib import contextmanager
import signal

class Interrupt:
  def __init__(self):
    self.interrupt = False
  
def signal_handler(signal, frame):
  interrupt.interrupt = True
  print('Terminating...')

@contextmanager
def graceful_exit():
  original_sigint_handler = signal.getsignal(signal.SIGINT)
  interrupt = Interrupt()
  signal.signal(signal.SIGINT, signal_handler)
  try:
    yield interrupt
  except:
    raise
  finally:
    signal.signal(signal.SIGINT, original_sigint_handler)

# Get links to New York Times articles for all tweets.
import psycopg2
conn = psycopg2.connect(host = 'localhost', database = 'nytpopular')
with conn.cursor() as cursor:
    cursor.execute('SELECT id, url FROM tweets;')
    allURLs = cursor.fetchall()
conn.close()
urlStack = list(reversed(allURLs))

In [ ]:
# Launch FireFox with Selenium

import os
import re
import time
import json
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import urlparse

firefoxProfile = '/Users/dilwong/Library/Application Support/Firefox/Profiles/aiyantvn.selenium' # This is my FireFox profile for selenium
driver = webdriver.Firefox(firefox_profile=firefoxProfile)

In [ ]:
# Scrape the NYT articles and save (1) the HTML and (2) a JSON file containing the title, summary paragraph, article text, full URL, # of comments, and whether the page has audio/video

with graceful_exit() as interrupt: # Note that CTRL + C will also close FireFox...
    while True:
        try:
            id_, url = urlStack.pop()
        except IndexError:
            print('DONE!!!')
            break
        driver.get(url)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Scroll to bottom to load entire page
        fullHTML = driver.page_source
        with open(f'ScrappedData/nytArticles/fullHTML/{id_}.html', 'w') as file:
            file.write(fullHTML)
        fullURL = driver.current_url
        parsedURL = urlparse(fullURL)
        if 'nytimes.com' not in parsedURL.hostname: # If the link is not a NYT article, skip it.
            continue
        if 'https://www.nytimes.com/interactive/2021/us/covid-cases.html' in fullURL: # If the link is the COVID case graph, skip it.
            time.sleep(2)
            continue
        if 'https://www.nytimes.com/interactive/2022/upshot/wordle-bot.html' in fullURL: # Skip Wordle
            time.sleep(2)
            continue
        if 'https://www.nytimes.com/puzzles/spelling-bee' in fullURL: # Skip Spelling Bee
            time.sleep(2)
            continue
        if 'https://www.nytimes.com/newsletters/the-daily' in fullURL: # Skip The Daily
            time.sleep(2)
            continue
        soup = BeautifulSoup(fullHTML)
        if parsedURL.path.split('/')[1] == 'live':
            liveSection = soup.find_all(attrs={'data-url': re.compile(parsedURL.fragment)})
            if liveSection:
                titleText = ' '.join([elem.text for elem in liveSection[0].find_all(class_='css-16ra6z9')])
                summaryText = None
                articleText = ' '.join([elem.text for elem in liveSection[0].find_all(class_='evys1bk0')])
                hasVideo = bool(liveSection[0].find_all('video'))
                hasAudio = bool(liveSection[0].find_all('audio'))
            else:
                #titleText = ' '.join([elem.text for elem in soup.find_all(class_='e1h9rw200')])
                #summaryText = ' '.join([elem.text for elem in soup.select('.css-54b9nt.evys1bk0')])
                #articleText = ' '.join([elem.text for elem in soup.select('.live-blog-post-content.evys1bk0')])
                continue
        else:
            titleText = ' '.join([elem.text for elem in soup.find_all(class_='e1h9rw200')])
            if titleText == '':
                titleText = ' '.join([elem.text for elem in soup.find_all(class_='edye5kn2')])
            if titleText == '':
                titleText = ' '.join([elem.text for elem in soup.find_all(class_='_7503bcc6')])
            if titleText == '':
                titleText = ' '.join([elem.text for elem in soup.select('h1[data-scp="title hidden"]')])
            if titleText == '':
                titleText = ' '.join([elem.text for elem in soup.select('h1[itemprop="headline"]')])
            if titleText == '':
                titleText = ' '.join([elem.text for elem in soup.select('header.title-header > h1')])
            summaryText = ' '.join([elem.text for elem in soup.find_all(class_='e1wiw3jv0')])
            if summaryText == '':
                summaryText = ' '.join([elem.text for elem in soup.find_all(class_='edye5kn3')])
            if summaryText == '':
                summaryText = ' '.join([elem.text for elem in soup.select('header.title-header > p.g-dek')])
            if summaryText == '':
                summaryText = None
            articleText = (' '.join([elem.text for elem in soup.find_all(class_='g-body')]).replace('\n','').replace('\t','')
                            + ' ' + ' '.join([elem.text for elem in soup.find_all(class_='evys1bk0')])).strip()
            if articleText == '':
                articleText = (' '.join([elem.text for elem in soup.select("[data-testid='lede'] > p")])
                            + ' ' + ' '.join([elem.text for elem in soup.select("[data-gtm-element='intro'] > p")])
                            + ' ' + ' '.join([elem.text for elem in soup.select("div[role='tabpanel'] p")])).strip()
            if articleText == '':
                articleText = ' '.join([elem.text for elem in soup.select('div.blurb')]).replace('\n', ' ')
            if articleText == '':
                articleText = ' '.join([elem.text for elem in soup.select('p.css-fytyl0')])
            articleText = articleText.replace('\xa0',' ')
            hasVideo = bool(soup.find_all('video'))
            hasAudio = bool(soup.find_all('audio'))
        if parsedURL.path.split('/')[1] == 'recipes':
            titleText = soup.select('.recipe-title')[0].text.strip()
            articleText = ' '.join([elem.text for elem in soup.select('.topnote > p')]) + ' ' + ' '.join([elem.text for elem in soup.select('ol.recipe-steps')]).replace('\n',' ')
        if (titleText == '') or (articleText == ''):
            if parsedURL.path.split('/')[1] == 'video': # For video-only articles
                if articleText == '':
                    articleText = None
                if titleText == '':
                    titleText = None
            elif parsedURL.path.split('/')[1] == 'interactive':
                try:
                    articleText = (' '.join([elem.text for elem in soup.select('p.css-1xh7jop')]) + ' ' + ' '.join([elem.text for elem in soup.select('p.css-1jeom2t')])).strip()
                    possibleTitle = soup.select('h2.css-6fr66q')
                    if possibleTitle:
                        titleText = possibleTitle[0].text
                    possibleTitle = soup.select('h2.css-1ifcf0b')
                    if possibleTitle:
                        titleText = possibleTitle[0].text
                except:
                    print(f'Stopped at {id_}, {url} because either titleText or articleText is missing.')
                    break
                if (titleText == '') or (articleText == ''):
                    print(f'Stopped at {id_}, {url} because either titleText or articleText is missing.')
                    break
            else:
                print(f'Stopped at {id_}, {url} because either titleText or articleText is missing.')
                break
        nComment = soup.find(class_ = 'css-1dtr3u3') # Are there comments?
        if nComment is not None:
            nComment = nComment.text
            if nComment == '+':
                nComment = 0
            else:
                if nComment[-1] == 'k': # If there are comments, how many?
                    nComment = int(float(nComment[:-1]) * 1000)
                else:
                    nComment = int(nComment)
        try:
            nComment_alt = soup.select('button#comments-speech-bubble-bigBottom')[0].text.split(' ')[1] # More accurate comment count
            if nComment_alt == '+':
                nComment = 0
            else:
                nComment = int(nComment_alt)
        except:
            pass
        if nComment is None:
            try:
                switched_iframe = False
                frame = driver.find_elements(By.CSS_SELECTOR, 'div#disqus_thread > iframe[src*="https://disqus.com/embed/comments/"]')[0]
                driver.switch_to.frame(frame)
                switched_iframe = True
                newsoup = BeautifulSoup(driver.page_source)
                commentCount = newsoup.find(class_='comment-count').text.split()
                if commentCount[1] == 'comments':
                    nComment = int(commentCount[0])
            except:
                pass
            finally:
                nComment = None
                if switched_iframe:
                    driver.switch_to.parent_frame()
        if nComment is None:
            try:
                allTab = soup.select('#all-tab > .nytc---notessection---threadCount')
                if allTab:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(2)
                    fullHTML = driver.page_source
                    soup = BeautifulSoup(fullHTML)
                    allTab = soup.select('#all-tab > .nytc---notessection---threadCount')
                    nComment = int(allTab[0].text)
            except:
                pass
        jsonObject = {
                        'title': titleText,
                        'summary': summaryText,
                        'article': articleText,
                        'full_url': fullURL,
                        'comments': nComment,
                        'has_video': hasVideo,
                        'has_audio': hasAudio
        }
        with open(f'ScrappedData/nytArticles/articleText/{id_}.json', 'w') as file:
            json.dump(jsonObject, file)
        time.sleep(random.randrange(5,15)) # Wait 5-15 seconds before fetching next article. People probably don't like their websites being scraped, so this is out of politeness.
        if interrupt.interrupt:
            break

## Loading the scrapped NYT articles into a postgreSQL database

In [ ]:
import psycopg2
from psycopg2 import sql
from collections import OrderedDict
import json
import os

conn = psycopg2.connect(host = 'localhost', database = 'nytpopular')
conn.autocommit = True
cursor = conn.cursor()

# Create tweets table
cursor.execute('''CREATE TABLE articles (
	id BIGINT PRIMARY KEY,
    title VARCHAR(255),
    summary TEXT,
    article TEXT,
    url VARCHAR(500),
    comments INT,
    video BOOLEAN,
    audio BOOLEAN
);''')

def insertArticleintoDB(jsonObject, cursor):
    fields = OrderedDict()
    fields['id'] = jsonObject['id']
    fields['title'] = jsonObject['title']
    fields['summary'] = jsonObject['summary']
    fields['article'] = jsonObject['article']
    fields['url'] = jsonObject['full_url']
    fields['comments'] = jsonObject['comments']
    fields['video'] = 'TRUE' if jsonObject['has_video'] else 'FALSE'
    fields['audio'] = 'TRUE' if jsonObject['has_audio'] else 'FALSE'
    keys, items = zip(*fields.items())
    keys = sql.SQL(',').join(map(sql.Identifier, keys))
    items = sql.Literal(items)
    command = sql.SQL('INSERT INTO articles ({}) VALUES {} ON CONFLICT (id) DO NOTHING;').format(keys, items)
    cursor.execute(command)

articleFilenames = os.listdir('ScrappedData/nytArticles/articleText/')

for filename in articleFilenames:
    if '.json' in filename:
        with open('ScrappedData/nytArticles/articleText/' + filename) as f:
            jsonObject = json.load(f)
            jsonObject['id'] = int(filename.split('.')[0])
            insertArticleintoDB(jsonObject, cursor)

cursor.close()
conn.close()

In [ ]:
# Just in case, purge any articles that don't have an associated tweet. This shouldn't happen.
conn = psycopg2.connect(host = 'localhost', database = 'nytpopular')
conn.autocommit = True
with conn.cursor() as cursor:
    cursor.execute('''DELETE FROM articles
                    WHERE id IN (SELECT articles.id AS a_id FROM tweets RIGHT JOIN articles ON tweets.id = articles.id WHERE tweets.id is NULL);''')
conn.close

## Get article metadata through the New York Times API

The New York Times API has some interesting metadata, including keywords for each article.<br>
If one wanted, one could also get vocabulary/tags from the Semantic API and user comments from the Community API.<br>
Here, I only fetch information from the Archive API.

In [ ]:
import time
import json
import requests

apiKey = '' # Replace with NYT API key
months = [12, 1, 2, 3, 4]
years = [2019] + [2022] * 4

for month, year in zip(months, years):
    req = requests.get(f'https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={apiKey}')
    with open(f'nyt_metadata_{month:02}_{year}.json', 'w', encoding = 'utf-8') as f:
        json.dump(req.json(), f, indent = 4)
    time.sleep(5)

Insert the metadata into POSTGRES database.

In [ ]:
import psycopg2
from psycopg2 import sql
from collections import OrderedDict
import json
import os

conn = psycopg2.connect(host = 'localhost', database = 'nytpopular')
conn.autocommit = True
cursor = conn.cursor()

cursor.execute('''CREATE TABLE metadata (
	id VARCHAR(255) PRIMARY KEY,
    abstract TEXT,
    url VARCHAR(255),
    lead_paragraph TEXT,
    title VARCHAR(255),
    date TIMESTAMPTZ,
    document_type VARCHAR(100),
    news_desk VARCHAR(100),
    section VARCHAR(100),
    subsection VARCHAR(100),
    word_count SMALLINT,
    keywords TEXT
);''')

def insertMetadataintoDB(jsonObject, cursor):
    fields = OrderedDict()
    fields['id'] = jsonObject['_id']
    fields['abstract'] = jsonObject['abstract']
    fields['url'] = jsonObject['web_url']
    fields['lead_paragraph'] = jsonObject['lead_paragraph']
    fields['title'] = jsonObject['headline']['main']
    fields['date'] = jsonObject['pub_date'].replace('T',' ')
    fields['document_type'] = jsonObject['document_type']
    fields['news_desk'] = jsonObject['news_desk']
    fields['section'] = jsonObject['section_name']
    fields['subsection'] = jsonObject['subsection_name'] if 'subsection_name' in jsonObject else None
    fields['word_count'] = jsonObject['word_count']
    fields['keywords'] = '|'.join([entry['value'] for entry in jsonObject['keywords']]) or None
    keys, items = zip(*fields.items())
    keys = sql.SQL(',').join(map(sql.Identifier, keys))
    items = sql.Literal(items)
    command = sql.SQL('INSERT INTO metadata ({}) VALUES {} ON CONFLICT (id) DO NOTHING;').format(keys, items)
    cursor.execute(command)

articleFilenames = os.listdir('ScrappedData/nytMetadata/')

for filename in articleFilenames:
    if '.json' in filename:
        with open('ScrappedData/nytMetadata/' + filename, 'r') as f:
            wholeObject = json.load(f)
            for jsonObject in wholeObject['response']['docs']:
                insertMetadataintoDB(jsonObject, cursor)

cursor.close()
conn.close()

In [ ]:
# Add a table that maps tweet IDs (from Twitter) to NYT URIs (from the metadata).

import psycopg2

conn = psycopg2.connect(host = 'localhost', database = 'nytpopular')
conn.autocommit = True
cursor = conn.cursor()

sqlStatement = '''CREATE TABLE id_map AS
                SELECT articles.id tweet_id, metadata.id uri_id, articles.url article_url, metadata.url metadata_url FROM articles
                LEFT JOIN metadata ON articles.url LIKE metadata.url || '%';
                '''
cursor.execute(sqlStatement)
cursor.execute('ALTER TABLE id_map ADD PRIMARY KEY (tweet_id);')

cursor.close()
conn.close()